In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain
from langchain import OpenAI

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
import os
import getpass
import pickle
import time

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

OpenAI API Key:··········


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    separators=['\n\n', '\n', '.', ','],
    chunk_size=1000
)

In [ ]:
data_pdfs = os.listdir()[1:-1]
data_pdfs

['Financial Planning & Analysis and Performance Management.pdf',
 'Personal Financial Planning - GRAD - 2021 August.pdf',
 'technical-analysis-and-stock-market-profits.pdf',
 'hand_book.pdf',
 '13022023_150309_102120474.pdf',
 'L-G-0014439947-0045930201.pdf',
 'Personal Financial Planning Guide.pdf',
 'TA_wrkbk.pdf',
 'echap01.pdf']

In [ ]:
all_pages = []

for x in data_pdfs:
  loader = PyPDFLoader(x)
  pages = loader.load_and_split()
  all_pages = all_pages + pages

In [ ]:
docs = []
for x in all_pages:
  doc = text_splitter.split_documents([x])
  docs = docs + doc

In [ ]:
len(docs)

5678

In [ ]:
len(all_pages)

2184

In [ ]:
embeddings = OpenAIEmbeddings()

In [ ]:
objPKL = []
i = 100
while i<=5600:
    vectorstore_openai = FAISS.from_documents(docs[i-100:i], embeddings)
    # with open('embeddings.pkl', "wb") as f:
    #   pickle.dump(vectorstore_openai, f)
    objPKL.append(vectorstore_openai)
    i = i+100
    time.sleep(20)

In [ ]:
for x in objPKL[1:]:
  with open('embeddings.pkl', "ab") as f:
    pickle.dump(x.serialize_to_bytes(), f)

In [ ]:
pkl = objPKL[0].serialize_to_bytes()

In [ ]:
with open('embeddings.pkl', "wb") as f:
  pickle.dump(pkl, f)

In [ ]:
db = FAISS.deserialize_from_bytes(
    embeddings=embeddings, serialized=pkl
)  # Lo

In [ ]:
with open('embeddings.pkl', "rb") as f:
  vectorstore = pickle.load(f)

In [ ]:
vectorstore = FAISS.deserialize_from_bytes(
    embeddings=embeddings, serialized=vectorstore
)

In [ ]:
llm = OpenAI(temperature=0.9, max_tokens=500)

In [ ]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorstore.as_retriever())

In [ ]:
result = chain({"question": "where can i invest money"}, return_only_outputs=True)

In [ ]:
result["answer"]

'\nInvestment options can be found at www.WileyFinance.com.\n'